In [1]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pytorch_tabnet.tab_model import TabNetRegressor
import pandas as pd
import numpy as np
import seaborn as sb
print(torch.version.cuda)

12.1


In [2]:
merged_data = pd.read_csv('data/merged_data.csv')
merged_data = merged_data.drop(columns=['Set'])
ml_data = merged_data.dropna()
ml_data

,Year,Month,town,flat_type,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price_adjusted,average_storey
456621,2017,1,SEMBAWANG,4 ROOM,100.0,Model A,1999,81.333333,345907.470070,5.0
456622,2017,1,SEMBAWANG,4 ROOM,86.0,Model A2,2000,82.500000,314930.681705,2.0
456623,2017,1,QUEENSTOWN,5 ROOM,110.0,Improved,2006,88.916667,888001.266448,20.0
456624,2017,1,QUEENSTOWN,5 ROOM,111.0,Improved,2003,85.666667,826047.689719,20.0
456626,2017,1,QUEENSTOWN,4 ROOM,92.0,Model A,2013,95.250000,727954.526565,29.0
...,...,...,...,...,...,...,...,...,...,...
628402,2024,1,HOUGANG,4 ROOM,90.0,Model A,2003,78.416667,495000.000000,2.0
628403,2024,1,HOUGANG,4 ROOM,94.0,Model A,2019,94.916667,630000.000000,8.0
628404,2024,1,HOUGANG,4 ROOM,93.0,Model A,2018,93.833333,528000.000000,2.0
628405,2024,1,HOUGANG,4 ROOM,93.0,Model A,2019,94.666667,630000.000000,8.0


In [3]:
ml_data['flat_model'].unique()

array(['Model A', 'Model A2', 'Improved', 'Apartment',
       'Premium Apartment', 'Adjoined flat', 'New Generation', 'DBSS',
       'Standard', 'Simplified', 'Maisonette', 'Model A-Maisonette',
       'Type S1', 'Terrace', 'Type S2', 'Improved-Maisonette',
       'Premium Maisonette', 'Multi Generation', 'Premium Apartment Loft',
       '2-room', '3Gen'], dtype=object)

In [4]:
area_dict = {'QUEENSTOWN': 1, 'SERANGOON': 2, 'PASIR RIS': 3, 'TAMPINES': 4, 'MARINE PARADE': 5, 'KALLANG/WHAMPOA': 6, 'JURONG WEST': 7, 'WOODLANDS': 8, 'YISHUN': 9, 'TOA PAYOH': 10, 'BUKIT PANJANG': 11, 'BISHAN': 12, 'BUKIT BATOK': 13, 'BEDOK': 14, 'BUKIT MERAH': 15, 'ANG MO KIO': 16, 'HOUGANG': 17, 'JURONG EAST': 18, 'GEYLANG': 19, 'CHOA CHU KANG': 20, 'CLEMENTI': 21, 'CENTRAL AREA': 22, 'BUKIT TIMAH': 23, 'SENGKANG': 24, 'SEMBAWANG': 25, 'PUNGGOL': 26}
model_type_dict = {'Model A': 1, 'Standard': 2, 'Apartment': 3, 'Improved': 4, 'New Generation': 5, 'Simplified': 6, 'Terrace': 7, 'Adjoined flat': 8, 'Maisonette': 9, 'Multi Generation': 10, 'Premium Apartment': 11, 'Model A-Maisonette': 12, 'Improved-Maisonette': 13, 'Premium Maisonette': 14, '2-room': 15, 'Model A2': 16, 'DBSS': 17, 'Type S1': 18, 'Type S2': 19, 'Premium Apartment Loft': 20, '3Gen': 21}
flat_type_dict = {'5 ROOM': 1, 'EXECUTIVE': 2, '3 ROOM': 3, '4 ROOM': 4, '2 ROOM': 5, 'MULTI-GENERATION': 6, '1 ROOM': 7}


In [5]:
ml_data['town'] = ml_data['town'].map(area_dict)
ml_data['flat_type'] = ml_data['flat_type'].map(flat_type_dict)
ml_data['flat_model'] = ml_data['flat_model'].map(model_type_dict)

ml_data

C:\Users\ashwi\AppData\Local\Temp\ipykernel_1392\4018356990.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ml_data['town'] = ml_data['town'].map(area_dict)
C:\Users\ashwi\AppData\Local\Temp\ipykernel_1392\4018356990.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ml_data['flat_type'] = ml_data['flat_type'].map(flat_type_dict)
C:\Users\ashwi\AppData\Local\Temp\ipykernel_1392\4018356990.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

,Year,Month,town,flat_type,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price_adjusted,average_storey
456621,2017,1,25,4,100.0,1,1999,81.333333,345907.470070,5.0
456622,2017,1,25,4,86.0,16,2000,82.500000,314930.681705,2.0
456623,2017,1,1,1,110.0,4,2006,88.916667,888001.266448,20.0
456624,2017,1,1,1,111.0,4,2003,85.666667,826047.689719,20.0
456626,2017,1,1,4,92.0,1,2013,95.250000,727954.526565,29.0
...,...,...,...,...,...,...,...,...,...,...
628402,2024,1,17,4,90.0,1,2003,78.416667,495000.000000,2.0
628403,2024,1,17,4,94.0,1,2019,94.916667,630000.000000,8.0
628404,2024,1,17,4,93.0,1,2018,93.833333,528000.000000,2.0
628405,2024,1,17,4,93.0,1,2019,94.666667,630000.000000,8.0


In [6]:
X = ml_data.drop(columns=['resale_price_adjusted'])
y = ml_data['resale_price_adjusted']


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_tensor = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

y_train_tensor = y_train.values.reshape(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled)

tabnet_model = TabNetRegressor(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2)
)

c:\Users\ashwi\Documents\SC1015_Mini_Proj\.mini\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [10]:
tabnet_model.fit(X_train_tensor, y_train.values.reshape(-1, 1), max_epochs=400, patience=50,batch_size=128,virtual_batch_size=1280,loss_fn=torch.nn.functional.mse_loss,num_workers=8)

c:\Users\ashwi\Documents\SC1015_Mini_Proj\.mini\Lib\site-packages\pytorch_tabnet\abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 299431787411.549|  0:00:35s
epoch 1  | loss: 274789126833.5164|  0:01:10s
epoch 2  | loss: 238734752403.835|  0:01:45s
epoch 3  | loss: 191563162851.1751|  0:02:20s
epoch 4  | loss: 138146461049.86383|  0:02:55s
epoch 5  | loss: 86733972227.71011|  0:03:29s
epoch 6  | loss: 50327671850.23859|  0:04:04s
epoch 7  | loss: 32740270643.942|  0:04:39s
epoch 8  | loss: 24978983827.5496|  0:05:14s
epoch 9  | loss: 11218161988.495|  0:05:49s
epoch 10 | loss: 6651233016.00892|  0:06:24s
epoch 11 | loss: 6211269109.44036|  0:06:59s
epoch 12 | loss: 6071765597.03902|  0:07:33s
epoch 13 | loss: 5965304515.2107|  0:08:08s
epoch 14 | loss: 5859899054.37681|  0:08:43s
epoch 15 | loss: 5711305706.02452|  0:09:18s
epoch 16 | loss: 5569333116.43256|  0:09:53s
epoch 17 | loss: 5476128123.29096|  0:10:28s
epoch 18 | loss: 5482257551.83946|  0:11:02s
epoch 19 | loss: 5407518381.52062|  0:11:37s
epoch 20 | loss: 5274924190.10925|  0:12:12s
epoch 21 | loss: 5320870514.7291|  0:12:47s
epoch 22

KeyboardInterrupt: 